# Kinematic Data Augmention

This notebook demonstrates a data augmentation process for kinematic data, including the estimation of additional joint positions, angles, and distances, using geometric methods for both relative and absolute values, with the assumption that the performer is in an upright position.

### Required Modules

In [ ]:
import os
import cv2
import math
import pandas as pd
import numpy as np

## Indexing by Seconds

For consistency and efficiency, we index the frames by seconds using the frames-per-second (fps) value. This allows us to divide the data into equal time intervals and facilitates the computation of features such as distances and velocities.

In [ ]:
kd = pd.read_csv('./data/interim/contemporary_001/landmarks_rel.csv')
fps = int(kd['fps'][0])
kd = kd[::fps].reset_index(drop=True)
kd.tail(5)

## Augmenting Joint Positions

MediaPipe's current implementation involves using 32 landmarks to identify joints (as shown in a reference picture). In order to gain a more comprehensive understanding of body alignment and posture during performances, we have estimated additional joint positions, which can be helpful in achieving this goal.

![Image](https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png)

### Forehead Joint

This computation estimates the position of the forehead joint using the average position of the outer corners of both eyes in both relative and absolute kinematic data.

In [5]:
kd.loc[:, 'forehead_x'] = (kd['left_eye_outer_x'] + kd['right_eye_outer_x']) / 2
kd.loc[:, 'forehead_y'] = (kd['left_eye_outer_y'] + kd['right_eye_outer_y']) / 2
kd.loc[:, 'forehead_z'] = (kd['left_eye_outer_z'] + kd['right_eye_outer_z']) / 2

### Torso Joint

This computation estimates the location of the torso joint by averaging the positions of the left and right shoulders and left and right hips.

In [6]:
kd.loc[:, 'torso_x'] = (kd['left_shoulder_x'] + kd['right_shoulder_x'] + kd['left_hip_x'] + kd['right_hip_x']) / 4
kd.loc[:, 'torso_y'] = (kd['left_shoulder_y'] + kd['right_shoulder_y'] + kd['left_hip_y'] + kd['right_hip_y']) / 4
kd.loc[:, 'torso_z'] = (kd['left_shoulder_z'] + kd['right_shoulder_z'] + kd['left_hip_z'] + kd['right_hip_z']) / 4

## Augmenting Joint Data with Angles, Distance, and Velocity
We can extract valuable information about body alignment and posture during a performance by augmenting the joint data with joint angles, distances, and velocities. To compute joint angles, we use the dot product of the joint vectors and the arccosine function to convert it into degrees. For distances, we use the Euclidean distance between the joint positions. To compute velocities, we take the difference in distance between consecutive frames and multiply it by the frames-per-second (fps) value. These computed features are based on the connections between limbs in the human body.

And here are the formulas for angle, distance, and velocity in LaTeX:

Joint angle: $\theta = \cos^{-1}\left(\frac{\vec{v_1}\cdot\vec{v_2}}{\left|\vec{v_1}\right|\left|\vec{v_2}\right|}\right)$

Joint distance: $d = \left|\vec{v_1} - \vec{v_2}\right|$

Joint velocity: $v = (d_t - d_{t-1}) \cdot \text{fps}$

where $\vec{v_1}$ and $\vec{v_2}$ are the joint vectors at time $t$ and $d_t$ and $d_{t-1}$ are the distances between the joints at time $t$ and $t-1$, respectively.

Source: [Vector Operations](https://www.khanacademy.org/math/algebra-home/alg-vectors)

### Defining Connected Joints
The joint pairs defined represent connections between limbs in the human body.

In [7]:
joint_pairs = [('right_shoulder', 'right_elbow'), ('right_elbow', 'right_wrist'), ('right_hip', 'right_knee'), ('right_knee', 'right_ankle'), ('right_ankle', 'right_foot_index'), ('left_shoulder', 'left_elbow'), ('left_elbow', 'left_wrist'), ('left_hip', 'left_knee'), ('left_knee', 'left_ankle'), ('left_ankle', 'left_foot_index'), ('forehead', 'torso')]

### Computing Angles and Magnitude

In [8]:
for joint in joint_pairs:
    angles = []
    distances = []
    velocities = []
    for i in range(len(kd)):
        joint1 = np.array([kd[f"{joint[0]}_x"].iloc[i], kd[f"{joint[0]}_y"].iloc[i], kd[f"{joint[0]}_z"].iloc[i]])
        joint2 = np.array([kd[f"{joint[1]}_x"].iloc[i], kd[f"{joint[1]}_y"].iloc[i], kd[f"{joint[1]}_z"].iloc[i]])     
        dot_product = np.dot(joint1, joint2)
        magnitude1 = np.linalg.norm(joint1)
        magnitude2 = np.linalg.norm(joint2)
        distance = np.linalg.norm(joint1 - joint2)
        angle = np.degrees(np.arccos(dot_product / (magnitude1 * magnitude2)))

        angles.append(angle)
        distances.append(distance)

        if i == 0:
            velocity = 0
            velocities.append(velocity)
        else:
            velocity = (distances[i] - distances[i-1]) * fps
            velocities.append(velocity)

    kd[f"a_{joint[0]}_{joint[1]}"] = angles
    kd[f"d_{joint[0]}_{joint[1]}"] = distances
    kd[f"v_{joint[0]}_{joint[1]}"] = velocities

## Rate of Change

The rate of change of a quantity measures how quickly that quantity is changing over time. It can be approximated by computing the difference between consecutive values of the quantity and dividing by the time difference between those values.

\begin{equation}
\text{rate of change}[i] = \frac{x[i] - x[i-1]}{t[i] - t[i-1]}
\end{equation}

[Reviewer on Khan Academy](https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:functions/x2f8bb11595b61c86:average-rate-of-change/v/introduction-to-average-rate-of-change)

In [9]:
joints = ["forehead", "torso", "right_shoulder", "left_shoulder", "right_elbow", "left_elbow", "right_knee", "left_knee", "right_ankle", "left_ankle"]
joints_x = [joint + "_x" for joint in joints]
joints_y = [joint + "_y" for joint in joints]
joints_z = [joint + "_z" for joint in joints]

for joint_x, joint_y, joint_z in zip(joints_x, joints_y, joints_z):
    diff_x = kd[joint_x].diff().fillna(0)
    diff_y = kd[joint_y].diff().fillna(0)
    diff_z = kd[joint_z].diff().fillna(0)
    kd[f"diff_{joint_x}"] = diff_x
    kd[f"diff_{joint_y}"] = diff_y
    kd[f"diff_{joint_z}"] = diff_z
    
kd.head()

,frame,fps,nose_x,nose_y,nose_z,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_x,left_eye_y,...,diff_right_knee_z,diff_left_knee_x,diff_left_knee_y,diff_left_knee_z,diff_right_ankle_x,diff_right_ankle_y,diff_right_ankle_z,diff_left_ankle_x,diff_left_ankle_y,diff_left_ankle_z
0,0,30.0,0.501249,0.210489,-0.084651,0.508947,0.194841,-0.062590,0.514194,0.195405,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,30.0,0.501941,0.211275,-0.245253,0.509505,0.196352,-0.219250,0.514590,0.196742,...,0.000324,-3.278255e-06,0.003548,0.063210,-0.004626,0.005318,0.049378,-0.000087,0.016001,0.047973
2,2,30.0,0.502662,0.211565,-0.249403,0.510098,0.196954,-0.222894,0.515015,0.197282,...,-0.001048,1.150370e-05,0.002482,0.001534,-0.002545,0.005571,-0.002750,-0.000019,0.006170,0.005876
3,3,30.0,0.503204,0.211633,-0.246048,0.510507,0.197138,-0.219161,0.515307,0.197445,...,-0.000439,1.132488e-06,0.002056,-0.001476,-0.000776,0.003955,-0.003464,-0.000049,0.003285,-0.000676
4,4,30.0,0.503533,0.211636,-0.246408,0.510845,0.197157,-0.219035,0.515560,0.197458,...,-0.000938,-5.960464e-08,0.001808,0.000249,-0.000197,0.001393,0.004212,-0.000009,0.001040,0.006928


## Save Processed DataFrames

In [10]:
kd.to_csv('./data/processed/contemporary_001_all.csv')

## Absolute Values

In [ ]:
h, w = 360, 640
kd_abs = kd.copy()
kd_abs.loc[:, kd_abs.columns.str.endswith('_x')] *= w
kd_abs.loc[:, kd_abs.columns.str.endswith('_y')] *= h
kd_abs.loc[:, kd_abs.columns.str.endswith('_z')] = h - kd_abs.loc[:, kd_abs.columns.str.endswith('_z')]
kd_abs.to_csv('./data/processed/contemporary_001_abs.csv', index=False)

## Batch Processing

For more details about the data augmentation processing, please refer to the code in [/src/data/augmentation.py](https://github.com/kayesokua/gestures/blob/main/src/data/augmentation.py)

In [1]:
from src.data.augmentation import batch_data_augmentation
batch_data_augmentation("./data/processed")

contemporary_007 has been processed!
contemporary_006 has been processed!
contemporary_004 has been processed!
contemporary_005 has been processed!
contemporary_001 has been processed!
classical_ballet_001 has been processed!
contemporary_002 has been processed!
contemporary_003 has been processed!
chinese_fan_001 has been processed!
contemporary_008 has been processed!
filipino_folk_001 has been processed!
Elapsed time: 0.717 seconds


## Summary
* The kinematic data has been indexed by second based on the fps value.
* Augmentation of the data includes new joint positions (torso, forehead), angles, distances, velocities of connected joints (connected limbs), and rate of change added for 10 joints (with the potential to add more).
* Further iterations and analysis may lead to changes in the data augmentation process.
   
## Resources
For a comprehensive list of resources, [please see here](https://github.com/kayesokua/gestures/blob/main/references/README.md).